# Mandelbrot set visualisation

The Mandelbrot set is the subset of the complex numbers $c$ such that the sequence
\begin{equation}
z_0=0 \hspace{1cm} z_{n+1}=z_n^2+c
\end{equation}
remains bounded. It can be shown that if $|z_n|\geq2$ for some $n$, then the sequence is unbounded (i.e. $2$ is an escape radius for that sequence).

<span style="font-size: x-small">See also: the [documentation](../doc/index.html) of module `fractal`.</span>

In [1]:
%autoreload 2

import logging
logger = logging.getLogger()

from numpy import array, zeros, ones, linspace, sum, abs, floor, square, sqrt, newaxis, nan, seterr
from fractal import FractalAnimation
from itertools import islice

In [2]:
def mandelbrot(N,bounds):
    xb, yb = bounds
    r = (xb[1]-xb[0])/(yb[1]-yb[0])
    Nx = int(sqrt(N/r)); Ny = int(N/Nx)
    tmd,tmap = zeros((2,Nx,Ny),float)
    c = array(linspace(xb[0],xb[1],Ny),dtype=complex)[newaxis,:]+1.j*array(linspace(yb[0],yb[1],Nx),dtype=complex)[:,newaxis]
    z = zeros((Nx,Ny),dtype=complex)
    undecided = ones((Nx,Ny),bool)
    sel = zeros((Nx,Ny),bool)
    n = 0
    seterr(invalid='ignore')
    while True:
        square(z,z)
        z += c
        sel[...] = abs(z)>=2.
        z[sel] = nan
        undecided[sel] = False
        n += 1
        tmd[...] = -tmap
        tmd[undecided] += 1
        tmap += tmd/n
        yield tmap

def MandelbrotAnimation(ax,N=250000,bounds=((-2.5,1.),(-1.,1.)),itermax=100):
    img = ax.imshow(zeros((1,1),float),vmin=0.,vmax=1.,origin='lower',extent=bounds[0]+bounds[1])
    def frames(bounds):
        return islice(((tmap,bounds) for tmap in mandelbrot(N,bounds)),itermax)
    def func(frm,interrupt=False):
        tmap,bounds = frm
        if interrupt:
            img.set_array(tmap)
            img.set_extent(bounds[0]+bounds[1])
        img.changed()
        return img,
    return FractalAnimation(ax,func=func,frames=frames,init_func=(lambda: None),interval=100,repeat=False)

Launcher
--------

* Open a new zoom level by selecting a rectangle with the mouse (button click on one corner, keep pressed, and release on opposite corner).

* Navigate through the different zoom levels using the arrow keys on the keyboard (up or right = forward, down or left = backward)

* The zoom level is indicated in the top right corner. The precision is indicated in the top left corner. At deeper zoom levels, details start to appear at higher precision levels.

In [3]:
logger.setLevel(logging.INFO)
def test(**ka):
    from matplotlib.pyplot import figure, show
    fig = figure(figsize=(8,8))
    ax = fig.add_axes((0,0,1,1),xticks=(),yticks=())
    MandelbrotAnimation(ax,**ka)
    show()
test(N=160000,itermax=1000)